# Shaping de la forward curve markit

## Forward Curve TGP

In [61]:
# on recupere la donnee en time serie de KDB
from kdb import *
import pandas as pd
from ratios.ratio import Ratio
from ratios.scope import Scope
from ratios.ratio import Ratio
from ratios.profiles.filters.filter import Filter
from ratios.smoothings.smoothing import Smoothing
from ratios.smoothings.integral_cubic_spline import IntegralCubicSpline
from ratios.profiles.baseload import Baseload
from ratios.profiles.filters.hours import Hours
from ratios.profiles.filters.months import Months
from ratios.profiles.filters.years import Years
from ratios.profiles.off_peak import OffPeak, OverlappingIndexesPeakAndOffPeak
from ratios.profiles.peak import Peak
from ratios.scope import Scope

# donnee markit du 21
data_fr_month_markit = Shape('2022-09-21', "FRANCE_BL", "Month")["Markit"]
data_fr_qua_markit = Shape('2022-09-21', "FRANCE_BL", "Quarter")["Markit"]
data_fr_cal_markit = Shape('2022-09-21', "FRANCE_BL", "Year")["Markit"]

# donnee tgp du 21
data_fr_month_tgp = Shape('2022-09-21', "FRANCE_BL", "Month")["TGP"]
data_fr_qua_tgp = Shape('2022-09-21', "FRANCE_BL", "Quarter")["TGP"]
data_fr_cal_tgp = Shape('2022-09-21', "FRANCE_BL", "Year")["TGP"]

# donnee markit du 22
data_fr_month_1_markit = Shape('2022-09-22', "FRANCE_BL", "Month")["Markit"]
data_fr_qua_1_markit = Shape('2022-09-22', "FRANCE_BL", "Quarter")["Markit"]
data_fr_cal_1_markit = Shape('2022-09-22', "FRANCE_BL", "Year")["Markit"]

# donnee tgp du 22
data_fr_month_1_tgp = Shape('2022-09-21', "FRANCE_BL", "Month")["TGP"]
data_fr_qua_1_tgp = Shape('2022-09-21', "FRANCE_BL", "Quarter")["TGP"]
data_fr_cal_1_tgp = Shape('2022-09-21', "FRANCE_BL", "Year")["TGP"]

Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-22
Data retrieved from kdb for 2022-09-22
Data retrieved from kdb for 2022-09-22
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21
Data retrieved from kdb for 2022-09-21


In [62]:
print(data_fr_month_markit.loc['2023-01-01':'2023-12-01'].mean())
print(data_fr_qua_markit.loc['2023-01-01':'2023-12-01'].mean())
print(data_fr_cal_markit.loc['2023-01-01':'2023-12-01'].mean())

576.0744009291666
575.263517395
574.29248498


In [63]:
print(data_fr_month_tgp.loc['2023-01-01':'2023-12-01'].mean())
print(data_fr_qua_tgp.loc['2023-01-01':'2023-12-01'].mean())
print(data_fr_cal_tgp.loc['2023-01-01':'2023-12-01'].mean())

578.6007577663864
577.5461848674657
576.0


In [94]:
# On calcule ici le ratio de prix de markit
ratio_quar_cal = Ratio(Baseload(data_fr_qua_markit),  #on peut utiliser les filtres par annees, quarters, mois .filter_(Years([2022,2023,2024])
                        line_scope=[Scope.YEAR_NUMBER],
                        column_scope=[Scope.QUARTER]) \
    .compute(numerator_computing_scope=[Scope.YEAR_NUMBER, Scope.QUARTER]).evaluate()

ratio_months_quar = Ratio(Baseload(data_fr_month_markit),  #on peut utiliser les filtres par annees, quarters, mois .filter_(Years([2022,2023,2024])
                        line_scope=[Scope.YEAR_NUMBER,Scope.QUARTER],
                        column_scope=[Scope.MONTH_NUMBER]) \
    .compute(numerator_computing_scope=[Scope.YEAR_NUMBER,Scope.QUARTER, Scope.MONTH_NUMBER]).evaluate()

#print(ratio_quar_cal.matrix_rate)  
#print(ratio_months_quar.matrix_rate)


Start Date         1         2         3         4
Start Date                                        
2023        1.701835  0.663457  0.641876  0.992832
2024        2.005698  0.534793  0.514228  0.945280
2025        1.314209  0.745116  0.781102  1.159574
2026        1.345002  0.738729  0.759044  1.157224
2027        1.338433  0.760353  0.765365  1.135848
2028        1.215827  0.831685  0.849878  1.102609
2022             NaN       NaN       NaN  1.000000
Start Date                   1         2         3         4         5   \
Start Date Start Date                                                     
2022       4                NaN       NaN       NaN       NaN       NaN   
2023       1           1.178576  1.110627  0.710797       NaN       NaN   
           2                NaN       NaN       NaN  1.075137  0.933886   
           3                NaN       NaN       NaN       NaN       NaN   
           4                NaN       NaN       NaN       NaN       NaN   
2024       1    

In [70]:
# maintenant on observe une serie de prix cal, on essaie d'en determiner les prix quarters en utilisant les ratios calcules avec les prix de la veille
# plot du graph 
plt.figure(figsize = (30, 10), dpi = 650)
plt.style.use('seaborn')

#prix recalcule
a = Baseload(data_fr_cal_1_markit.to_period("Q").resample("H").ffill()).shape(ratio_quar_cal).data
a = a.resample("Q").mean()
a.index = a.index.to_timestamp()


plt.plot(a, color = "blue", label = "Prix recalcule")
plt.plot(data_fr_qua_tgp, color = "green", label = "Prix TGP")
plt.plot(data_fr_qua_1_markit, color = "indianred", label = "Prix Markit")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval = 240))
plt.title(f"Prix Quarter recalcule vs benchmarck")
plt.xlabel("Maturity")
plt.ylabel("Prix")
plt.xticks(rotation=90)
plt.legend();

In [82]:
# maintenant on observe une serie de prix cal, on essaie d'en determiner les prix quarters en utilisant les ratios calcules avec les prix de la veille
# plot du graph 
plt.figure(figsize = (30, 10), dpi = 650)
plt.style.use('seaborn')

#prix recalcule
a = Baseload(data_fr_qua_1_markit.to_period("M").resample("D").ffill()).shape(ratio_months_quar).data
a = a.resample("M").mean()
a.index = a.index.to_timestamp()


plt.plot(a, color = "blue", label = "Prix recalcule")
plt.plot(data_fr_month_1_markit, color = "green", label = "Prix TGP")
plt.plot(data_fr_month_1_tgp, color = "indianred", label = "Prix Markit")
plt.gca().xaxis.set_major_locator(mdates.DayLocator(interval = 240))
plt.title(f"Prix Quarter recalcule vs benchmarck")
plt.xlabel("Maturity")
plt.ylabel("Prix")
plt.xticks(rotation=90)
plt.legend();